In [2]:
# import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.simplefilter(action="ignore")

In [3]:
# rows and columns settings
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

In [4]:
# loading the dataset
df = pd.read_csv("persona.csv")

In [22]:
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [5]:
# writing a function to look at the overall picture
def check_df(dataframe, head=5):
    print('##################### Shape #####################')
    print(dataframe.shape)
    print('##################### Types #####################')
    print(dataframe.dtypes)
    print('##################### Head #####################')
    print(dataframe.head(head))
    print('##################### Tail #####################')
    print(dataframe.tail(head))
    print('##################### NA #####################')
    print(dataframe.isnull().sum())
    print('##################### Quantiles #####################')
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [6]:
check_df(df)

##################### Shape #####################
(5000, 5)
##################### Types #####################
PRICE       int64
SOURCE     object
SEX        object
COUNTRY    object
AGE         int64
dtype: object
##################### Head #####################
   PRICE   SOURCE   SEX COUNTRY  AGE
0     39  android  male     bra   17
1     39  android  male     bra   17
2     49  android  male     bra   17
3     29  android  male     tur   17
4     49  android  male     tur   17
##################### Tail #####################
      PRICE   SOURCE     SEX COUNTRY  AGE
4995     29  android  female     bra   31
4996     29  android  female     bra   31
4997     29  android  female     bra   31
4998     39  android  female     bra   31
4999     29  android  female     bra   31
##################### NA #####################
PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64
##################### Quantiles #####################
        count  mean   std   min    

In [7]:
# average price analysis
agg_df = df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"]).agg({"PRICE": "mean"}).sort_values("PRICE", ascending=False)

In [8]:
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
usa,android,male,36,59.00
bra,android,male,46,59.00
fra,android,female,24,59.00
usa,ios,male,32,54.00
deu,android,female,36,49.00


In [9]:
# Converting the names in the index to variable names.
agg_df = agg_df.reset_index()

In [10]:
# age categorization
agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], bins=[0, 18, 23, 30, 40, 70],
                           labels=['0_18', '19_23', '24_30', '31_40', "41_" + str(agg_df["AGE"].max())])

In [11]:
# put the VALUES of the variables COUNTRY, SOURCE, SEX and age next to each other and combine them with an underscore.
agg_df["customers_level_based"] = [row[0].upper() + "_" +
                                   row[1].upper() + "_" +
                                   row[2].upper() + "_" +
                                   row[5].upper() for row in agg_df.values]

In [12]:
# remove other variables
agg_df = agg_df[["customers_level_based", "PRICE"]]

In [13]:
agg_df["customers_level_based"].value_counts().head()

customers_level_based
BRA_ANDROID_FEMALE_24_30    7
BRA_ANDROID_MALE_24_30      7
USA_ANDROID_MALE_41_66      7
USA_ANDROID_MALE_24_30      7
USA_IOS_FEMALE_24_30        7
Name: count, dtype: int64

In [14]:
# deduplication of segments
agg_df = agg_df.groupby("customers_level_based").agg({"PRICE": "mean"})

In [15]:
# Converting the names in the index to variable names.
agg_df = agg_df.reset_index()

In [16]:
# Segmentation by PRICE variable
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])

In [17]:
agg_df.head()

,customers_level_based,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,35.65,B
1,BRA_ANDROID_FEMALE_19_23,34.08,C
2,BRA_ANDROID_FEMALE_24_30,33.86,C
3,BRA_ANDROID_FEMALE_31_40,34.90,B
4,BRA_ANDROID_FEMALE_41_66,36.74,A


In [18]:
# 33 year old Turkish woman using ANDROID
new_user_1 = "TUR_ANDROID_FEMALE_31_40"

In [19]:
# Average Revenue Prediction
agg_df[agg_df["customers_level_based"] == new_user_1]

,customers_level_based,PRICE,SEGMENT
72,TUR_ANDROID_FEMALE_31_40,41.83,A


In [20]:
# 50 year old Brazilian man using IOS
new_user_2 = "BRA_IOS_MALE_41_66"

In [21]:
# Average Revenue Prediction
agg_df[agg_df["customers_level_based"] == new_user_2]

,customers_level_based,PRICE,SEGMENT
19,BRA_IOS_MALE_41_66,31.08,D


In [23]:
# Import the necessary libraries for clustering.
from sklearn.cluster import KMeans

# Assuming 'agg_df' is your aggregated DataFrame which has:
# - "customers_level_based": the unique customer identifier;
# - "PRICE": the average price (or revenue);
# - "SEGMENT": the label from your rule-based quantile segmentation.
# For example, agg_df might look like:
#   customers_level_based         PRICE SEGMENT
# 0  BRA_ANDROID_FEMALE_0_18       35.65       B
# 1  BRA_ANDROID_FEMALE_19_23      34.08       C
# ... (and so on)

# ============================================
#Apply KMeans Clustering on the PRICE variable.
# ============================================
# We choose 4 clusters (to mirror the four segments we created: A, B, C, and D).
kmeans = KMeans(n_clusters=4, random_state=42)
agg_df['KMeans_Segment'] = kmeans.fit_predict(agg_df[['PRICE']])

# ============================================
#Compare the Rule-Based Segments vs. KMeans Segments.
# ============================================
# A cross-tabulation provides a quick look to see how the two segmentations align.
comparison = pd.crosstab(agg_df['SEGMENT'], agg_df['KMeans_Segment'])
print("Comparison of Rule-Based Segments vs KMeans Segments:")
print(comparison)

# ============================================
#Analyzing New User Predictions.
# ============================================
# For example, let’s analyze a new customer using the existing identifiers:
new_user_1 = "TUR_ANDROID_FEMALE_31_40"

# Get the average price for this new user from the aggregated dataframe.
# (Make sure that the new user's identifier exists in your agg_df.)
new_user_price = agg_df.loc[agg_df["customers_level_based"] == new_user_1, "PRICE"].values[0]

# Predict the cluster (segment) for the new user based on their PRICE.
new_user_kmeans_segment = kmeans.predict([[new_user_price]])[0]

print(f"New user '{new_user_1}' has an average PRICE of {new_user_price}.")
print(f"KMeans assigns this new user to cluster: {new_user_kmeans_segment}")

Comparison of Rule-Based Segments vs KMeans Segments:
KMeans_Segment   0   1   2   3
SEGMENT                       
D                0  10  18   0
C                0   0  27   0
B               19   0   8   0
A               14   0   0  13
New user 'TUR_ANDROID_FEMALE_31_40' has an average PRICE of 41.83333333333333.
KMeans assigns this new user to cluster: 3


In [24]:
from sklearn.cluster import DBSCAN

# Let's assume agg_df already contains:
# - "customers_level_based": the unique identifier,
# - "PRICE": the average revenue,
# - "SEGMENT": the rule-based segmentation (e.g., A, B, C, D).

# ============================================
#Apply DBSCAN Clustering on the PRICE feature.
# ============================================
# Because PRICE is a one-dimensional feature, we reshape it to a 2D array.
# The eps and min_samples parameters are crucial. Here we try eps=3 and min_samples=5, 
# but you may need to tune these based on your data distribution.
dbscan = DBSCAN(eps=3, min_samples=5)
agg_df['DBSCAN_Segment'] = dbscan.fit_predict(agg_df[['PRICE']])

# ============================================
# Compare the Rule-Based Segments vs DBSCAN Clusters
# ============================================
# Create a cross-tabulation to see how the rule-based segments align with the DBSCAN clusters.
comparison_dbscan = pd.crosstab(agg_df['SEGMENT'], agg_df['DBSCAN_Segment'])
print("Comparison of Rule-Based Segments vs DBSCAN Segments:")
print(comparison_dbscan)

# ============================================
#Heuristic New User Assignment with DBSCAN
# ============================================
# Unlike KMeans, DBSCAN does not have a predict method.
# One common workaround for one-dimensional data is:
#   - Compute the average PRICE for each DBSCAN cluster (ignoring noise labeled as -1).
#   - Assign a new user to the cluster whose centroid is closest to the new user's PRICE.

# Example for a new user
new_user_1 = "TUR_ANDROID_FEMALE_31_40"

# Get the new user's PRICE from the aggregated data.
new_user_price = agg_df.loc[agg_df["customers_level_based"] == new_user_1, "PRICE"].values[0]

# Compute the mean PRICE for each DBSCAN cluster (ignoring noise points: DBSCAN_Segment = -1).
clusters_centroids = agg_df[agg_df['DBSCAN_Segment'] != -1].groupby('DBSCAN_Segment')['PRICE'].mean()

# Find the cluster centroid closest to the new user's PRICE.
dbscan_assigned_cluster = (clusters_centroids - new_user_price).abs().idxmin()

print(f"\nNew user '{new_user_1}' has an average PRICE of {new_user_price:.2f}.")
print(f"Heuristically, DBSCAN assigns this new user to cluster: {dbscan_assigned_cluster}")


Comparison of Rule-Based Segments vs DBSCAN Segments:
DBSCAN_Segment  -1   0
SEGMENT               
D                2  26
C                0  27
B                0  27
A                1  26

New user 'TUR_ANDROID_FEMALE_31_40' has an average PRICE of 41.83.
Heuristically, DBSCAN assigns this new user to cluster: 0
